In [3]:
using FermiCG
using Printf
using Test
using JLD2 

@load "../test/_testdata_cmf_h12_64bit.jld2"

clustered_ham = FermiCG.extract_ClusteredTerms(ints, clusters)
cluster_ops = FermiCG.compute_cluster_ops(cluster_bases, ints);
FermiCG.add_cmf_operators!(cluster_ops, cluster_bases, ints, d1.a, d1.b);

v_bst = FermiCG.BSTstate(clusters, FockConfig(init_fspace), cluster_bases)
v_tps = FermiCG.TPSCIstate(clusters, FockConfig(init_fspace))
FermiCG.eye!(v_tps)

s_bst = FermiCG.build_compressed_1st_order_state(v_bst, cluster_ops, clustered_ham, nbody=4, thresh=1e-24)
s_tps = FermiCG.open_matvec_thread(v_tps, cluster_ops, clustered_ham, thresh=1e-32)

@show size(s_bst);
@show size(s_tps);





 Remove duplicates
 Number of terms reduced from  2525 to  1105
IDX001:DIM0016:001|002|
IDX002:DIM0016:003|004|
IDX003:DIM0256:005|006|007|008|
IDX004:DIM0016:009|010|
IDX005:DIM0016:011|012|
 Number of tasks:                                         661
 Compute tasks:                                    
 Time building compressed vector:                    0.161381 seconds   6.24e-02 Gb GC: 3.8e-02 sec
 Add results together:                               0.024873 seconds   1.74e-02 Gb GC: 0.0e+00 sec
 In open_matvec_thread
 Setup threaded jobs:                                0.000209 seconds (1.58 k allocations: 145.469 KiB)
 Number of jobs:    525
 Number of threads: 1
 Compute matrix-vector:                              0.016953 seconds (78.87 k allocations: 6.587 MiB)
 Now collect thread results :                        0.000206 seconds (1.06 k allocations: 64.359 KiB)
size(s_bst) = (16278, 1)
size(s_tps) = (49378, 1)


In [173]:
e_tpsci, v_tpsci = FermiCG.tpsci_ci(v_tps, cluster_ops, clustered_ham, thresh_cipsi=1e-4, thresh_foi=1e-6, thresh_asci=-1)
e_pt = FermiCG.compute_pt2_energy(v_tpsci, cluster_ops, clustered_ham, thresh_foi=1e-8, verbose=0);

 ci_vector     : (1, 1)
 thresh_cipsi  : 0.0001
 thresh_foi    : 1.0e-6
 thresh_asci   : -1
 thresh_var    : nothing
 max_iter      : 10
 conv_thresh   : 0.0001
 nbody         : 4
 incremental   : true
 ci_conv       : 1.0e-5
 ci_max_iter   : 50
 ci_max_ss_vecs: 12
 davidson      : false
 max_mem_ci    : 20.0
 threaded      : true


     Selected CI Iteration:    1 epsilon:   0.00010000
 Memory needed to hold full CI matrix:   0.00000001 (Gb) Max allowed:  20.00000000 (Gb)

 |== Tensor Product State CI =======================================
 Hamiltonian matrix dimension =     1: 
 Build full Hamiltonian matrix with dimension:       0.000170 seconds (522 allocations: 14.922 KiB)
 Now diagonalize
 Diagonalization time:                               0.000002 seconds
(1, 1)(1, 1)
 Compute S2 expectation values:                      0.000119 seconds (221 allocations: 8.062 KiB)
  Root       Energy           S2
     1 -18.31710895   0.00000000
 ==============================================

# BST PT2

In [122]:
e_var = FermiCG.ci_solve(v_bst, cluster_ops, clustered_ham);
e_pt = FermiCG.compute_pt2_energy(v_bst, cluster_ops, clustered_ham, thresh_foi=1e-6, max_iter=100, tol=1e-9, verbose=0);

 |== BST CI ========================================================
 Solve CI with # variables:                                 1
 Cache Hamiltonian:                                  0.020021 seconds (3.98 k allocations: 7.854 MiB, 89.75% gc time)
 Iter:   1 SS: 1    E:  -18.31710895*  R: 0.0e+00*  LinDep: 0.0e+00* 
 Diagonalization time:                               0.002138 seconds
 Compute <S^2>:                                      0.000496 seconds (1.29 k allocations: 158.047 KiB)
  Root       Energy           S2
     1 -18.31710895   0.00000000
 ==================================================================|
 |== Compute PT2 Energy ============================================
 H0          : Hcmf
 max_iter    : 100
 nbody       : 4
 thresh_foi  : 1.0e-6
 max_number  : nothing
 tol         : 1.0e-9
 opt_ref     : true
 verbose     : 0

 Length of Reference:                                       1
 Solve zeroth-order problem:                       
 |== BST CI ================

In [189]:
e_var, v = FermiCG.ci_solve(v_bst, cluster_ops, clustered_ham);

s_bst = FermiCG.build_compressed_1st_order_state(v, cluster_ops, clustered_ham, nbody=4, thresh=1e-33)
    
# FermiCG.project_out!(s_bst, v, thresh=1e-32)
# @show size(s_bst)
# s_bst = FermiCG.compress(s_bst, thresh=1e-32)
# @show size(s_bst)
v_pt, e_pt = FermiCG.hylleraas_compressed_mp2(s_bst, v, cluster_ops, clustered_ham, tol=1e-8, verbose=1);

 |== BST CI ========================================================
 Solve CI with # variables:                                 1
 Cache Hamiltonian:                                  0.003338 seconds (3.98 k allocations: 7.854 MiB)
 Iter:   1 SS: 1    E:  -18.31710895*  R: 0.0e+00*  LinDep: 0.0e+00* 
 Diagonalization time:                               0.001389 seconds
 Compute <S^2>:                                      0.000427 seconds (1.29 k allocations: 158.047 KiB)
  Root       Energy           S2
     1 -18.31710895   0.00000000
 ==================================================================|
 Number of tasks:                                         661
 Compute tasks:                                    
 Time building compressed vector:                    0.211288 seconds   6.24e-02 Gb GC: 6.6e-02 sec
 Add results together:                               0.028415 seconds   1.76e-02 Gb GC: 0.0e+00 sec
 Length of input      FOIS:                             16382
 Compute <X|

In [6]:
e_pt, v_pt = FermiCG.do_fois_pt2(v_bst, cluster_ops, clustered_ham, thresh_foi=1e-3, max_iter=50, tol=1e-9);

 |== Solve for BST PT1 Wavefunction ================================
 H0          : Hcmf
 max_iter    : 50
 nbody       : 4
 thresh_foi  : 0.001
 tol         : 1.0e-9
 opt_ref     : true
 verbose     : true

 Length of Reference:                                       1
 Solve zeroth-order problem:                       
 |== BST CI ========================================================
 Solve CI with # variables:                                 1
 Cache Hamiltonian:                                  0.001821 seconds (3.98 k allocations: 7.854 MiB)
 Iter:   1 SS: 1    E:  -18.31710895*  R: 0.0e+00*  LinDep: 0.0e+00* 
 Diagonalization time:                               0.001210 seconds
 Compute <S^2>:                                      0.000684 seconds (1.29 k allocations: 158.047 KiB)
  Root       Energy           S2
     1 -18.31710895   0.00000000
 ==================================================================|
 Diagonalization time:                               0.006731 seco

# Pseudo-Canonicalization

In [199]:
e_var, v = FermiCG.ci_solve(v_bst, cluster_ops, clustered_ham);

s_bst = FermiCG.build_compressed_1st_order_state(v, cluster_ops, clustered_ham, nbody=4, thresh=1e-33)
    
FermiCG.pseudo_canon_pt1(s_bst, v, cluster_ops, clustered_ham, verbose=1);
# v_pt, e_pt = FermiCG.pseudo_canon_pt1(s_bst, v, cluster_ops, clustered_ham, tol=1e-8, verbose=1);


 |== BST CI ========================================================
 Solve CI with # variables:                                 1
 Cache Hamiltonian:                                  0.002702 seconds (3.98 k allocations: 7.854 MiB)
 Iter:   1 SS: 1    E:  -18.31710895*  R: 0.0e+00*  LinDep: 0.0e+00* 
 Diagonalization time:                               0.001114 seconds
 Compute <S^2>:                                      0.000510 seconds (1.29 k allocations: 158.047 KiB)
  Root       Energy           S2
     1 -18.31710895   0.00000000
 ==================================================================|
 Number of tasks:                                         661
 Compute tasks:                                    
 Time building compressed vector:                    0.163356 seconds   6.24e-02 Gb GC: 2.0e-02 sec
 Add results together:                               0.052782 seconds   1.76e-02 Gb GC: 2.3e-02 sec

 |...................................BST-PT2.............................

In [200]:
e_var, v = FermiCG.ci_solve(v_bst, cluster_ops, clustered_ham);

v = FermiCG.build_compressed_1st_order_state(v, cluster_ops, clustered_ham, nbody=4, thresh=1e-3)

e_var, v = FermiCG.ci_solve(v, cluster_ops, clustered_ham); 

s_bst = FermiCG.build_compressed_1st_order_state(v, cluster_ops, clustered_ham, nbody=4, thresh=1e-6)

@time v_pt, e_pt = FermiCG.hylleraas_compressed_mp2(s_bst, v, cluster_ops, clustered_ham, tol=1e-8, verbose=1);

 |== BST CI ========================================================
 Solve CI with # variables:                                 1
 Cache Hamiltonian:                                  0.001723 seconds (3.98 k allocations: 7.854 MiB)
 Iter:   1 SS: 1    E:  -18.31710895*  R: 0.0e+00*  LinDep: 0.0e+00* 
 Diagonalization time:                               0.001270 seconds
 Compute <S^2>:                                      0.000350 seconds (1.29 k allocations: 158.047 KiB)
  Root       Energy           S2
     1 -18.31710895   0.00000000
 ==================================================================|
 Number of tasks:                                         661
 Compute tasks:                                    
 Time building compressed vector:                    0.151262 seconds   4.52e-02 Gb GC: 2.8e-02 sec
 Add results together:                               0.006131 seconds   3.60e-03 Gb GC: 0.0e+00 sec
 |== BST CI ========================================================
 Solv

In [205]:
@time FermiCG.pseudo_canon_pt1(s_bst, v, cluster_ops, clustered_ham, verbose=1);


 |...................................BST-PT2............................................
 Compute <X|H|0>:                                   13.823067 seconds   2.77e+00 Gb
 Length of input      FOIS:                            109661
 State   1: E(PT2) corr:                          -0.00063280
  Root         E(0)         E(2)
     1 -18.32971586 -18.33034866
 State   1: E(PT2) corr:                          -0.00040021
  Root         E(0)         E(2)
     1 -18.32971586 -18.33011607
 ......................................................................................|
 18.885177 seconds (42.93 M allocations: 3.180 GiB, 38.23% gc time, 11.91% compilation time)


In [217]:
@time v_pt, e_pt = FermiCG.compute_pt1_wavefunction(s_bst, v, cluster_ops, clustered_ham, verbose=1);


 |...................................BST-PT2............................................
 Length of input      FOIS:                            109661
 Compute <X|H|0>:                                   11.336102 seconds   2.75e+00 Gb
 Compute <X|F|0>:                                    0.041119 seconds   1.51e-02 Gb
 State   1: E(PT2) corr:                          -0.00040339
  Root         E(0)         E(2)
     1 -18.32971586 -18.33011925
 ......................................................................................|
 13.794174 seconds (42.81 M allocations: 3.190 GiB, 25.66% gc time, 1.68% compilation time)


In [214]:
@time v_pt, e_pt = FermiCG.compute_pt1_wavefunction(v, cluster_ops, clustered_ham, verbose=1, thresh_foi=1e-6);